In [ ]:
texts = df.Text.tolist()

In [ ]:
import emot

emot_obj = emot.core.emot()
text = "I love python ☮ 🙂 ❤ :-) :-( :-)))"
print(emot_obj.emoji(text)["value"])


In [ ]:
texts

In [ ]:
import pickle
import os
def f(dict, path):
    path = os.path.join(path, dict)
    with open(path, 'rb') as f:
        return pickle.load(f)

path = os.path.join("emoji_data")
files = os.listdir(path)
loaded_dicts = [f(file, path) for file in files]

In [ ]:
dics_names = [
    "EMOJI_UNICODE",
    "UNICODE_EMOJI",
    "EMOJI_ALIAS_UNICODE",
    "UNICODE_EMOJI_ALIAS",
    "EMOTICONS_EMO",
]

In [ ]:
text_val = []
for text in texts:
    if emot_obj.emoji(text)["flag"]:
        val = emot_obj.emoji(text)["value"]
        text_val.append([text, val])



In [ ]:
emos = emot_obj.emoji(text)["value"]

In [ ]:
emos

In [ ]:
list(loaded_dicts[2].keys()).index(list(loaded_dicts[2].keys())[0])

In [ ]:
keys = list(loaded_dicts[2].keys())
in_test_emo = keys[0:7]
in_test_emo


In [ ]:
keys.index(in_test_emo[0])

In [ ]:
import numpy as np

In [ ]:
keys = np.array(list(loaded_dicts[2].keys()))


In [ ]:
def find_indeces(columns, values):
    columns = columns.flatten()
    inds = np.array([np.where(columns == searchval)[0] for searchval in values]).flatten()
    return inds

In [ ]:
inds = find_indeces(keys, in_test_emo)

In [ ]:
zeros = np.zeros((2,keys.shape[0]))

In [ ]:
zeros[0][inds] = 1

In [ ]:
zeros[0][0:10]

In [ ]:
vals = emot_obj.emoji(text)["value"]

In [ ]:
zeros[0][find_indeces(keys, vals)] = 1

In [ ]:
emojis_extraced = [emot_obj.emoji(text)["value"] for text in texts if emot_obj.emoji(text)["flag"]]
training_len = len(emojis_extraced)
encoded = np.zeros((training_len,keys.shape[0]))

In [ ]:
for row, emojis in zip(encoded, emojis_extraced):
    row[find_indeces(keys,emojis)] = 1

In [ ]:
encoded.shape

In [1]:
main = pd.read_csv("dataset_.csv")[["emoi","label"]]

dropped_empty = main[main.emoi != "[]"]
import ast
dropped_empty['emoi']=dropped_empty['emoi'].apply(ast.literal_eval)
emojis_set = dropped_empty.emoi.to_list()
encoded = np.zeros((len(emojis_set),keys.shape[0]))
for row, emojis in zip(encoded, emojis_set):
    row[find_indeces(keys,emojis)] = 1
encoded_df = pd.DataFrame(encoded)
dropped_empty.reset_index(drop=True, inplace=True)
dropped_empty.index
merged = dropped_empty.join(encoded_df)

NameError: name 'pd' is not defined